<a href="https://colab.research.google.com/github/ucheokechukwu/ml_tensorflow_deeplearning/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# introduction to NLP fundamentals in Tensorflow

NLP has the goal of deriving information out of natural langauge (could be sequence text or speech).

Another common term for NLP problems is sequence to sequence problmes (seq2seq).

In [1]:
## check for GPU
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
# get helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-03-07 05:10:56--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-03-07 05:10:56 (63.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset
Kaggle's introduction to NLP dataset. Text samples of tweets labelled as disaster or not disaster. 
- binary clssification
https://www.kaggle.com/c/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")


--2023-03-07 05:11:09--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.128, 142.251.111.128, 172.253.62.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2023-03-07 05:11:10 (70.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

to visualize our text samples, we first have to read them in. we can do so using Pandas for Python 

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][20]

'this is ridiculous....'

In [6]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)

In [7]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# what does the text dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# how many examples of each class are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# how many total samples
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# let's visualize some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
Owner of Chicago-Area Gay Bar Admits to Arson Scheme http://t.co/0TSlQjOKvh via @theadvocatemag #LGBT

---

Target: 0 (not real disaster)
Text:
Moved on to 'Bang Bang Rock and Roll' by @Art_Brut_ . It's been too long since I've played this one loud. ART BRUT TOP OF THE POPS.

---

Target: 1 (real disaster)
Text:
@stunckle @Gordon_R74 @crazydoctorlady ...I'm no expert but raw uranium and nuclear reactor fuel rods are 2 very different creatures...

---

Target: 0 (not real disaster)
Text:
Don't argue cheap now. You're better than that. ??

---

Target: 1 (real disaster)
Text:
Gold Coast tram hit by fallen powerlines: UP to 30 people have been evacuated from a tram on the Gold Coast af... http://t.co/xpPQnYHiWC

---



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                             train_df_shuffled["target"].to_numpy(),
                                                                             test_size=0.1,
                                                                             random_state=42)
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [13]:
# Check the first ten examples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with text problem, one of the first things you need to do is numerically encode the text.

Methods:

1. Tokenization - direct mapping of token (word or character to number) or one-hot encoding.

2 - Embedding - creating a matrix of feature vectors for each token. The size of the vector can be defined and this embedding, which is essentially a matrix of weights can be learned.

## Text vectorization (tokenization)

In [14]:
import tensorflow as tf
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import TextVectorization

In [15]:
# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None
                                    )

In [16]:
# find the average number of tokens (words) in the training tweets

In [17]:
len(train_sentences[0].split())


7

In [18]:

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [19]:
# set up text vectorization variables
max_vocab_length = 10000 #max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

# fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [20]:
sample_sentence="there is a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

* Note that the shape is (1,15) because we passed it in **1** sequence and **15** is because the max_length is 15.

In [21]:
text_vectorizer(["there is a man in my backyard!"])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  74,    9,    3,   89,    4,   13, 6143,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [22]:
random_sentence = random.choice(train_sentences)
print(f"Original text: \n{random_sentence}\n\n\nVectorized Version: {text_vectorizer([random_sentence])}")

Original text: 
@colinhoffman29 I hope he does. And I hope you die in the explosion too


Vectorized Version: [[  1   8 237  56 350   7   8 237  12 686   4   2 303 150   0]]


In [23]:
# get the unique words in the vocubalary
words_in_vocab = text_vectorizer.get_vocabulary() # get all the unique words in our training data
top_5_words = words_in_vocab[:10]
bottom_5_words = words_in_vocab[-10:]
print(f"Number of words in vocab: {len(words_in_vocab)} \n\n5 most common words: \n{top_5_words}\n\n5 least common words: \n{bottom_5_words}")
# [UNK] is unknown text, that is it's outside of 10000 words

Number of words in vocab: 10000 

5 most common words: 
['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

5 least common words: 
['painthey', 'painful', 'paine', 'paging', 'pageshi', 'pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Text vectorization (embedding)
`tf.keras.layers.Embedding`
turns positive integers into dense vectors of fixed size
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:
* `input_dim` - the size of our vocabulary
* `output_dim` - the size of the output embedding vector e.g. a value of 100 means each token gets represented by a vector of length 100
* `input_length` - the length of sequences passed into the embedding layer (in this case, it's 15)

In [24]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length, #set input shape
                             output_dim=128, #neural networks work best with numbers divisible by 8
                             input_length=max_length # how long is each input
)

In [25]:
# test on random sentences from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text: \n{random_sentence}\
n\nEmbedded version:")
# embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer(random_sentence))
sample_embed

Original text: 
The advantages apropos of in flames favorable regard mississauga ontario: pWHvGwaxn
Embedded version:


<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[ 0.00574607,  0.00982568,  0.00980378, ..., -0.00720889,
        -0.00415068,  0.00840224],
       [ 0.00668132, -0.04883543, -0.02191256, ..., -0.02630205,
         0.03629002, -0.01627064],
       [ 0.00668132, -0.04883543, -0.02191256, ..., -0.02630205,
         0.03629002, -0.01627064],
       ...,
       [-0.01913604,  0.00063217, -0.0436939 , ...,  0.03311293,
        -0.03767068, -0.01613659],
       [-0.01913604,  0.00063217, -0.0436939 , ...,  0.03311293,
        -0.03767068, -0.01613659],
       [-0.01913604,  0.00063217, -0.0436939 , ...,  0.03311293,
        -0.03767068, -0.01613659]], dtype=float32)>

In [26]:
sample_embed = tf.expand_dims(sample_embed, axis=0)

In [27]:
# check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00574607,  0.00982568,  0.00980378, -0.04440099, -0.01681957,
        -0.01338556, -0.02031734,  0.03458251,  0.01283895, -0.04322541,
         0.03638537, -0.03585386, -0.00268824, -0.04779773,  0.00198405,
        -0.0129557 ,  0.03089367, -0.03082103,  0.04153422, -0.03584001,
        -0.04880128, -0.03946282,  0.01219415,  0.00547151, -0.04575551,
        -0.043563  , -0.00792981, -0.01064067,  0.0055825 ,  0.00567733,
        -0.02224411,  0.01070451, -0.01826899,  0.03473668,  0.00857059,
        -0.01458249,  0.03732342, -0.04559366, -0.01550023,  0.00063896,
         0.04574713,  0.03937348,  0.00786572, -0.01780884,  0.00389009,
        -0.03908531,  0.00224327, -0.01637191, -0.00482257,  0.04349107,
        -0.01875731,  0.01968256,  0.0121888 , -0.03896987,  0.03932542,
         0.02696938,  0.01744603,  0.03776315, -0.03651906,  0.04914396,
        -0.02577795,  0.04437107,  0.02942128,  0.02727592, -0.00228705,
  

# Modelling our text dataset - running a series of experiments

It's time to start building a series of modelling experiments, starting with a baseline and moving on from there:

* Model 0: Naive Bayes (baseline)
* Model 1: feed-forward neural network (dense model)
* Model 2: LSTM model (long-short term memory) (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional LSTM model (RNN)
* Model 5: 1D Convolutional Neural network
* Model 6: Tensorflow Hub pretrained feature extracctor (using transfer learning for NLP)
* Model 7: same as 6 with 10% of the dataset

Method of approach: standard steps with modelling with tensorflow:
- prepare data -> build -> compile -> fit -> evaluate -> experiment and improve

## model 0 - getting a baseline
This will be our baseline model that serves as a benchmark for future experiments to build up. We're going to use `sklearn` Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers. 

* 🔑 It's common practice to use non-DL algorithm as a baseline because of their speed and later use DL to see how to improve upon them.

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


In [29]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
    ("clf", MultinomialNB()) # model the text using this classifier(clf)
])

# fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [33]:
# evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels) 
#.score is for sklearn what .evaluate is for tensorflow. the default evaluation metric for classification is accuracy

In [39]:
print(f"Our baseline score achieves an accuracy of {baseline_score*100:.2f}%")

Our baseline score achieves an accuracy of 79.27%


In [40]:
# make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [53]:
# Creating evaluation function
def evaluation (model, val_sentences, val_labels):
  """Function to return the evaluation metrics of a model 
  given the model and the validation data
  """
  from sklearn.metrics import recall_score, precision_score, classification_report
  accuracy = model.score(val_sentences, val_labels)
  predicted_labels = model.predict(val_sentences)
  precision = precision_score(val_labels, predicted_labels)
  recall = recall_score(val_labels, predicted_labels)
  report = classification_report(val_labels, predicted_labels)

  return accuracy, precision, recall, report

In [54]:
base_evaluation = evaluation(model_0, val_sentences, val_labels)
print(f"Accuracy is: {base_evaluation[0]*100:.2f}%. \nPrecision Score is:{base_evaluation[1]:.2f}\
\nRecall Score is: {base_evaluation[2]:.2f} \
\n\n\nClassification Report is {base_evaluation[3]}")

Accuracy is: 79.27%. 
Precision Score is:0.89
Recall Score is: 0.63 


Classification Report is               precision    recall  f1-score   support

           0       0.75      0.93      0.83       414
           1       0.89      0.63      0.73       348

    accuracy                           0.79       762
   macro avg       0.82      0.78      0.78       762
weighted avg       0.81      0.79      0.79       762



In [63]:
# Creating evaluation function
def calculate_results (y_true, y_preds):
  """Function to return the evaluation metrics of a model 
  given the model and the validation data
  """
  from sklearn.metrics import accuracy_score, precision_recall_fscore_support
  model_accuracy = accuracy_score(y_true, y_preds) *100
  
  model_prediction, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_preds,
                                                                                average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "prediction": model_prediction,
                   "recall": model_recall,
                   "f1_score": model_f1}

  return model_results

In [65]:
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'prediction': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}